In [44]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from fractions import Fraction
directory = 'lorenz/'


# dyn_lorenz = {"sigma":16.0, "R":45.92, "b": 4.0}
dyn_lorenz = {"sigma":16.0, "R":45.0, "b": 4}
x0 = np.array([10.0, 1.0, 0.0])
v1 = np.array([1.0, 0.0, 0.0])
v2 = np.array([0.0, 1.0, 0.0])
v3 = np.array([0.0, 0.0, 1.0])
x_dot = []
x_norm = []
le_array = []
# cum = np.array([0,0,0])

def lorenz (x0, dyn):
    sigma = dyn['sigma']
    R = dyn['R']
    b = dyn['b']
    x = x0[0]
    y = x0[1]
    z = x0[2]
    return np.array([sigma * (y - x), 
                     x * (R - z) - y, 
                     x * y - b * z])

# def linearized_lorenz (x0, dyn, y_lorenz):
#     sigma = dyn['sigma']
#     R = dyn['R']
#     b = dyn['b']
#     x = x0[0]
#     y = x0[1]
#     z = x0[2]
#     return np.array([sigma * (y - x), 
#                      (R - y_lorenz[2])*x - y - y_lorenz[0]*z,
#                       y_lorenz[1]*x + y_lorenz[0]*y - b*z])

# RungeKutta_linearized(dyn, linearized_f, dt, v1_prime[:, i-1], x[:, i-1])
def linearized_lorenz (x0, dyn, y_lorenz):
    sigma = dyn['sigma']
    R = dyn['R']
    b = dyn['b']
    x = y_lorenz[0]
    y = y_lorenz[1]
    z = y_lorenz[2]
    pre_dot = np.array([[-sigma, sigma, 0],
                        [R-z, -1, -x],
                        [y,x,-b]
                        ])
    af_dot = np.dot(pre_dot, x0)
    return af_dot


def RungeKutta (dyn, f, dt, x0):
    k1 = f(x0, dyn) #[x,y,z]*0.1 example
    k2 = f(x0 + 0.5*k1*dt,dyn)
    k3 = f(x0 + 0.5*k2*dt, dyn)
    k4 = f(x0 + k3*dt, dyn)
    
    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) *dt
    return x


def RungeKutta_linearized (dyn, f, dt, x0, y):
    k1 = f(x0, dyn,y) #[x,y,z]*0.1 example
    k2 = f(x0 + 0.5*k1*dt,dyn,y) 
    k3 = f(x0 + 0.5*k2*dt, dyn,y) 
    k4 = f(x0 + k3*dt, dyn,y) 
    
    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) * dt

    return x

def f_t (dyn, f, linearized_f, dt, x0, T):
    x = np.empty(shape=(len(x0),T))
    v1_prime = np.empty(shape=(len(x0),T))
    v2_prime = np.empty(shape=(len(x0),T))
    v3_prime = np.empty(shape=(len(x0),T))
    x[:, 0] = x0
    v1_prime[:, 0] = v1
    v2_prime[:, 0] = v2
    v3_prime[:, 0] = v3
    cum = np.array([0,0,0])
    for i in range(1,T):
        x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])
        
        v1_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v1_prime[:, i-1], x[:, i-1])
        v2_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v2_prime[:, i-1], x[:, i-1])
        v3_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v3_prime[:, i-1], x[:, i-1])
        
        
        norm1 = np.linalg.norm(v1_prime[:, i])
        v1_prime[:, i] = v1_prime[:, i]/norm1
        
        GSC1 = np.dot(v1_prime[:, i], v2_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i] - GSC1*v1_prime[:, i]
        
        norm2 = np.linalg.norm(v2_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i]/norm2
        
        GSC2 = np.dot(v3_prime[:, i], v1_prime[:, i])
        GSC3 = np.dot(v3_prime[:, i], v2_prime[:, i])
        
        v3_prime[:, i] = v3_prime[:, i] - GSC2*v1_prime[:, i] - GSC3*v2_prime[:, i]
        norm3 = np.linalg.norm(v3_prime[:, i])
        v3_prime[:, i] = v3_prime[:, i]/norm3
        
        cum = cum + np.log2(np.array([norm1,norm2,norm3]))
        le_array.append(cum/(i*dt))
#         if ( i % 100 == 0 ):
#             print ('log2:', np.log2(np.array([norm1,norm2,norm3]))/(i*dt))
#             print ('cum:', cum/(i*dt))
        
        
    return x, cum/(T*dt)



In [94]:
v, cum = f_t(dyn_lorenz, lorenz, linearized_lorenz, 0.001, np.array([  10, 10, 10]), 100)

In [91]:
cum

array([  5.90861827, -17.639022  , -18.26322597])

In [93]:
cum

array([ 25.88234361, -50.1629004 ,  -5.71307236])

In [95]:
cum

array([ 14.57997233, -21.93467829, -22.63892362])